# Redes Neuronales Convolucionales

Este tutorial es para raza bien pesada en machine learning :/

# Descripcion General

La clasificacion CIFAR-10 es un problema comun de benchmarck en machine learning. El problema es clasificar las images RGB de 32 X 32 pixeles en las siguientes 10 categorias: Avion, Carro, Pajaro, Gato, Venado, Perro, Rana, Caballo, Barco y Trailers.

![Image of Yaktocat](https://www.tensorflow.org/versions/master/images/cifar_samples.png)

# Objetivos

El objetivo de este tutorial es construir relativamente una pequena Red Neuronal Convolucional para reconocer imagenes. En el proceso de este tutorial:

1. Resaltar una organizacion canonical para la arquitectura de red, entrenamiento y evaluacion.
2. Provee un template para construir un modelo mas grande y sofisticado.

# Aspectos destacados del Tutorial

El tutorial CIFAR-10 demuestra varias construcciones importantes para un disenio grande y modelos mas sofisticados en TensorFlow:

* Componentes matematicos basicos incluidos tales como convolution, rectified linear activations, max pooling y local response normalization.
* Visualizacion de la red de actividades durante el entrenamiento incluyendo imagenes de entrada, perdidas y distribuciones de activacion y gradientes.
* Rutinas para calcular el moving average de los parametros de aprendisaje utilizando estos promedios durante la evaluacion para aumentar el rendimiento de prediccion.
* Implementacion de un learning rate schedule que sistematicamente decrese sobre el tiempo.
* Prefetching queues para los datos de entrada para isolar el modelo de la latencia del disco y el preprocesamiento de imagenes costosas.

# Arquitectura del Modelo

El modelo en este tutorial de CIFAR-10 es una arquitectura multicapas que consiste de convoluciones alternas y nonlinearities. Estas capas estan seguidas de capas fully connected dirigidas dentro del clasifiador softmax.

# Organizacion del Codigo

El codigo de este tutorial se encuentra en el github del proyecto.

Los archivos a utilizar son los siguientes:

* cifar10_input.py - Lee el formato nativo en binario del archivo CIFAR-10.
* cifar10.py - Construye el modelo CIFAR-10
* cifar10_train.py - Entrena el modelo CIFAR-10 en un CPU.
* cifar10_eval.py - Evalua el rendimiento de la exactitud de un Modelo CIFAR-10.


# Modelo CIFAR-10

La red CIFAR-10 esta contenida en gran parte en cifar10.py. El entrenamiento completo del grafo contiene aproximadamente 765 operaciones. El codigo se puede hacer mas reutilizable mediante la construccion en el grafo de los siguientes modulos:

1. Model inputs: inputs() y distorted_inputs() agrega operaciones que leen el preprocesamiento de las imagenes de CIFAR para la evaluacion y entrenamiento respectivamente.
2. Model prediction: inference() agrega operaciones que realizan inferencia por ejemplo clasificacion, en las imagenes dadas.
3. Model training: loss() y train() agrega operaciones que calculan la perdida, gradientes, actualizaciones de variables y visualizacion de recopilatorios.

# Entradas del Modelo

La parte de entrada del modelo es contruido por las funciones inputs() y distorted_inputs() lo cual leen imagenes de los archivos de datos binarios de CIFAR-10. Estos archivos contienen registros de bytes de longitud fija entonces se usa tf.FixedLengthRecordReader.

Las imagenes son procesadas de la siguiente manera:

* Las imagenes se recortan a un tamanio de 24 X 24 pixeles centralmente para la evaluacion aleatoria en el entrenamiento.
* Las imagenes estan aproximadamente whitened para hacer el modelo insensible en el rango dinamico.

Para entrenamiento adicionalmente se aplican series de distiorsiones aleatorias para incrementar artificialmente el tamanio del dataset:

* Se voltea aleatoriamente la imagen de izquierda a derecha.
* Se distorciona aleatoriamente el brillo de la imagen.
* Se distorciona aleatoriamente el contraste de la imagen.



# Prediccion del Modelo

La parte de la prediccion del modelo se contruye con la funcion de inference() la cual agrega operaciones para calcular los logits de las predicciones. Esta parte del modelo se organiza como sigue:

Nombre de la capa - Descripcion
conv1             - Convolution y rectified linear activation.
pool1             - max polling.
norm1             - local response normalization.
conv2             - convolution y rectified linear activation.
norm2             - local response normalization.
pool2             - max pooling.
local3            - fully connected layer with rectified linear activation.
local4            - fully connected layer with rectified linear activation.
softmax_linear    - Transformacion lineal que produce logits.

Aqui se presenta el grafo el cual fue generado por TensorBoard el cual describe la operacione de inferencia:

![Image of Yaktocat](https://www.tensorflow.org/versions/master/images/cifar_graph.png)

Las funciones inputs() y inference() proveen todos los componentes necesarios para realizar la evaluacion de un modelo. Ahora se cambiara el enfoque hacia las operaciones de construccion para la formacion de un modelo.


# Entrenamiento del Modelo

El metodo usual para entrenar una red es realizar la clasificacion N-Way la cual es una regresion logistica multinomial regresion softmax. La regresion Softmax aplica un softmax nonlinearity a la salida de la red y calcula la cross-entropy entre las predicciones normalizadas y un 1-hot codificado de la etiqueta. Para la regularizacion se aplica usualmente la perdida de weight decay para todas las variables de aprendisaje. El objetivo de la funcion para el modelo es la suma de perdida de cross-entropy y todos sus pesos en terminos de decadencia esto se regresa con la funcion loss().

Se puede visualizar la perdida total en el TensorBoard con un scalar_summary:

![Image of Yaktocat](https://www.tensorflow.org/versions/master/images/cifar_loss.png)

Se entrena el modelo usando el algoritmo de descenso de gradiente estandar con una taza de aprendisaje que decae exponencialmente sonbre el tiempo.

![Image of Yaktocat](https://www.tensorflow.org/versions/master/images/cifar_lr_decay.png)

La funcion train() agrega las operaciones necesarias para minimizar el objetivo tal que calcula el gradiente y actualiza las variables de aprendisaje. Regresa una operacion que ejecuta todos los calculos necesarios para entrenar y actualizar el modelo para un batch de imagenes.

# Inicializando y Entrenando el Modelo

Ya que se construyo el modelo se pasa a inicializarlo y ejecutar la operacion de entrenamiento con el script cifar10_train.py. De la manera siguiente

In [6]:
"""
import os
filepath = `/home/juanmanuel/Desktop/deep-learning/cifar10`
os.chdir(filepath)
%run cifar10_train.py
"""

'\nimport os\nfilepath = `/home/juanmanuel/Desktop/deep-learning/cifar10`\nos.chdir(filepath)\n%run cifar10_train.py\n'

La primera vez que se ejecuta el script antes presentado se descarga el dataset de CIFAR-10 el cual pesa alrededor de 160 mb.

El script reporta la perdida total cada 10 pasos y la velocidad en que se proceso el ultimo batch de datos. 

El primer batch de datos puede ser muy lento (bastante) como los hilos de preprocesamiento llenan la cola con 20000 imagenes de CIFAR-10.

La perdida reportada es el promedio de perdida de el batch mas reciente. Como nota hay que recordar que la perdida es la suma de cross-entropy y todos sus pesos en terminos de decadencia.

Se presenta las siguientes velocidades de procesamiento en un batch. Los numeros que se muestran abajo se obtuvieron con un Tesla K40c. Si se ejecuta en un CPU se espera un rendimiento lento (muy lento).

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.
* 2015-11-04 11:45:45.927302: step 0, loss = 4.68 (2.0 examples/sec; 64.221 sec/batch)
* 2015-11-04 11:45:49.133065: step 10, loss = 4.66 (533.8 examples/sec; 0.240 sec/batch)
* 2015-11-04 11:45:51.397710: step 20, loss = 4.64 (597.4 examples/sec; 0.214 sec/batch)
* 2015-11-04 11:45:54.446850: step 30, loss = 4.62 (391.0 examples/sec; 0.327 sec/batch)
* 2015-11-04 11:45:57.152676: step 40, loss = 4.61 (430.2 examples/sec; 0.298 sec/batch)
* 2015-11-04 11:46:00.437717: step 50, loss = 4.59 (406.4 examples/sec; 0.315 sec/batch)
...

Cuando se esta experimentando es molesto esperar que el primer paso de entrenamiento tarde tanto entonces se puede decrementar el parametro NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN en cifar10.py. el cual originalmente tiene 
el valor de 50000 lo estuve variando y le puse un valor para que "segun" fuera mucho mas rapido de 100 va cumplir 24 horas el entrenamiento y aun no termina ..

El archivo cifar10_train.py periodicamente guarda todos los parametros del modelo en archivos checkpoint pero estos no evaluan el modelo. Los archivos checkpoint seran usados por el archivo cifar10_eval.py para medir el rencimiento de la prediccion.

Ya que se ejecutaron los comandos antes mencionados se ha comenzado a entrenar un modelo CIFAR-10.

El texto que regresa el archivo cifar10_train.py provee una percepcion minima de como el modelo es entrenado. Si se quiere mas percepcion dentro del modelo durante el entrenamiento se pueden hacer las siguientes preguntas como por ejemplo

* La perdida decrese o solo es uido?
* El modelo suministra las imagenes apropiadas?
* Son los gradientes, activaciones y pesos razonables?
* Cual es la actual taza de aprendizaje?

Esta funcionalidad la provee TensorBoard mostrando los datos exportados periodicamente por cifar10_train.py atravez de SummaryWriter.

Por ejemplo se puede ver la distribucion de activaciones y grados de esparcimiento en local3 y sus caracteristicas durante el entrenamiento

![Image of Yaktocat](https://www.tensorflow.org/versions/master/images/cifar_sparsity.png)

![Image of Yaktocat](https://www.tensorflow.org/versions/master/images/cifar_activations.png)

# Evaluando el Modelo

Ahora se va evaluar que tan bueno es el modelo de entrenamiento  esto se realiza sobre un hold-out data set. El modelo se evalua mediante el script  cifar10_eval.py. Esto construye el modelo con la funcion de inference() y usa las 100000 imagnes e n la evaluacion del dataset CIFAR-10. Se calcula la precision en 1 en que la prediccion se empareje con la verdadera etiqueta de la imagen.

Para monitorear como el modelo mejora durante el entrenamiento el script de evaluacion se jecuta periodicamente en el ultimo archivo checkpoint que creo cifar10_train.py.

In [5]:
"""
import os
filepath = `/home/juanmanuel/Desktop/deep-learning/cifar10`
os.chdir(filepath)
%run cifar10_eval.py
"""

'\nimport os\nfilepath = `/home/juanmanuel/Desktop/deep-learning/cifar10`\nos.chdir(filepath)\n%run cifar10_eval.py\n'

Como nota se deben de ejecutar la evaluacion y el entrenamiento en diferentes GPUS ya que se puede quedar sin memoria durante la ejecucion.

La salida esperada debe ser como la siguiente

2015-11-06 08:30:44.391206: precision @ 1 = 0.860

El script simplemente regresa la precision cerca 1 periodicamente en este caso se regreso el 86% de exactitud. Ademas cifar10_eval.py exporta recopilaciones las cuales se pueden visualizar mediante TensorBoard. Estos recopilatorios proveen una percepcion adicional dentro del modelo de evaluacion.

El script de entrenamiento calcula la version de moving average para todas las variables de aprendisaje. El script de evaluacion sustituye todos los parametros de aprendisaje del modelo con la version de moving average. Esta sustitucion aumenta el rendimiento del modelo durante el tiempo de la evaluacion. 